In [8]:
# SETUP
!pip install lancedb open_clip_torch arxiv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=294e6b709120203187d4be5f4d77a118173b88d4feee49642595b3060423a544
  Stored in directory: /private/tmp/pip-ephem-wheel-cache-dyw0r0kc/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


## Explain Open CLIP and embeddings here

## Creating table from arxiv API

### Embedding Paper Summary using CLIP


In [19]:
import torch
import open_clip
import pandas as pd
from open_clip import tokenizer
from tqdm import tqdm
from collections import defaultdict
import arxiv
import lancedb

def embed_func_clip(text):
    model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
    tokenizer = open_clip.get_tokenizer('ViT-B-32')
    with torch.no_grad():
        text_features = model.encode_text(tokenizer(text))
    return text_features

### Create a DataFrame of the desired length

In [12]:
def get_arxiv_df(embed_func, length=10000):
    results = arxiv.Search(
      query= "cat:cs.AI OR cat:cs.CV OR cat:stat.ML",
      max_results = length,
      sort_by = arxiv.SortCriterion.Relevance,
      sort_order = arxiv.SortOrder.Descending
    ).results()
    df = defaultdict(list)
    for result in tqdm(results, total=length):
        try:
            df["title"].append(result.title)
            df["summary"].append(result.summary)
            df["authors"].append(str(result.authors))
            df["url"].append(result.entry_id)
            df["vector"].append(embed_func(result.summary).tolist()[0])

        except Exception as e:
            print("error: ", e)
    
    return pd.DataFrame(df)



### Create LanceDB Table

In [13]:
def create_table():
    db = lancedb.connect("db")
    df = get_arxiv_df(embed_func_clip)

    tbl = db.create_table("arxiv", data=df, mode="overwrite")

    return tbl

In [15]:
import lancedb
import build_table # helper local module

db = lancedb.connect("db")

if "arxiv" not in db.table_names():
    tbl = create_table()
else:
    tbl = db.open_table("arxiv")


## Semantic Search by concepts or summary

In [17]:
def search_table(query, embed_func=embed_func_clip, lim=3):
    db = lancedb.connect("db")
    tbl = db.open_table("arxiv")

    embs = embed_func(query)
    
    return tbl.search(embs.tolist()[0]).limit(3).to_df()


In [23]:
# MobileSAM paper abstract 2nd half
query = """
Many of such applications need to be run on resource-constraint edge devices,
like mobile phones. In this work, we aim to make SAM mobile-friendly by replacing the heavyweight
image encoder with a lightweight one. A naive way to train such a new SAM as in the original SAM
paper leads to unsatisfactory performance, especially when limited training sources are available. We
find that this is mainly caused by the coupled optimization of the image encoder and mask decoder,
motivated by which we propose decoupled distillation. Concretely, we distill the knowledge from
the heavy image encoder (ViT-H in the original SAM) to a lightweight image encoder, which can be
automatically compatible with the mask decoder in the original SAM. The training can be completed
on a single GPU within less than one day, and the resulting lightweight SAM is termed MobileSAM
which is more than 60 times smaller yet performs on par with the original SAM. For inference speed,
With a single GPU, MobileSAM runs around 10ms per image: 8ms on the image encoder and 4ms
on the mask decoder. With superior performance, our MobileSAM is around 5 times faster than the
concurrent FastSAM and 7 times smaller, making it more suitable for mobile applications. Moreover,
we show that MobileSAM can run relatively smoothly on CPU
"""


search_table(query)


,title,summary,authors,url,vector,score
0,Can SAM Count Anything? An Empirical Study on ...,Meta AI recently released the Segment Anything...,"[arxiv.Result.Author('Zhiheng Ma'), arxiv.Resu...",http://arxiv.org/abs/2304.10817v1,"[-0.023224907, 0.060689516, 0.14642626, -0.187...",35.039112
1,Communication-Computation Efficient Device-Edg...,"Device-edge co-inference, which partitions a d...","[arxiv.Result.Author('Xinjie Zhang'), arxiv.Re...",http://arxiv.org/abs/2108.13009v2,"[0.17511544, -0.14268509, 0.11111196, 0.025091...",35.435425
2,Saliency Prediction for Mobile User Interfaces,We introduce models for saliency prediction fo...,"[arxiv.Result.Author('Prakhar Gupta'), arxiv.R...",http://arxiv.org/abs/1711.03726v3,"[-0.23654917, 0.044955254, -0.060538974, -0.08...",36.065895


In [24]:

query = """Segment Anything Model (SAM) has attracted significant attention due to its impressive zero-shot
transfer performance and high versatility for numerous vision applications (like image editing with
fine-grained control). Many of such applications need to be run on resource-constraint edge devices,
like mobile phones. In this work, we aim to make SAM mobile-friendly by replacing the heavyweight
image encoder with a lightweight one. A naive way to train such a new SAM as in the original SAM
paper leads to unsatisfactory performance, especially when limited training sources are available. We
find that this is mainly caused by the coupled optimization of the image encoder and mask decoder,
motivated by which we propose decoupled distillation"""

search_table(query)


,title,summary,authors,url,vector,score
0,Attack-SAM: Towards Attacking Segment Anything...,Segment Anything Model (SAM) has attracted sig...,"[arxiv.Result.Author('Chenshuang Zhang'), arxi...",http://arxiv.org/abs/2305.00866v2,"[0.01118099, -0.0053622816, 0.0686877, -0.2128...",22.285025
1,Fast Segment Anything,The recently proposed segment anything model (...,"[arxiv.Result.Author('Xu Zhao'), arxiv.Result....",http://arxiv.org/abs/2306.12156v1,"[0.036730755, -0.067693904, -0.11393009, -0.11...",22.613714
2,SAM on Medical Images: A Comprehensive Study o...,The Segment Anything Model (SAM) made an eye-c...,"[arxiv.Result.Author('Dongjie Cheng'), arxiv.R...",http://arxiv.org/abs/2305.00035v1,"[-0.2498389, 0.11826321, -0.117645726, -0.1427...",23.955942


In [25]:
# Search via a concept you're reading
query = """
What is the general idea behin self-supervised learning.
"""

search_table(query)


,title,summary,authors,url,vector,score
0,Towards the Generalization of Contrastive Self...,"Recently, self-supervised learning has attract...","[arxiv.Result.Author('Weiran Huang'), arxiv.Re...",http://arxiv.org/abs/2111.00743v4,"[0.33684078, 0.12271507, -0.2530719, 0.3336045...",29.067186
1,Many-Goals Reinforcement Learning,All-goals updating exploits the off-policy nat...,"[arxiv.Result.Author('Vivek Veeriah'), arxiv.R...",http://arxiv.org/abs/1806.09605v1,"[0.081662394, -0.0060318857, -0.20181663, -0.1...",30.358921
2,Efficient Self-supervised Continual Learning w...,Inspired by the success of Self-supervised lea...,"[arxiv.Result.Author('Li Yang'), arxiv.Result....",http://arxiv.org/abs/2303.07477v1,"[0.16743122, 0.21459761, 0.09188169, 0.3162266...",32.915543


### Full Text Search


In [ ]:
# TODO